In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = (
    SparkSession.builder.master("local[2]")
    .appName("demo")
    .config(
        map={
            "spark.driver.memory": "2G",
            "spark.jars.packages": "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1,org.postgresql:postgresql:42.6.0",
            "spark.sql.extensions": "com.datastax.spark.connector.CassandraSparkExtensions",
            "spark.sql.catalog.comwatt": "com.datastax.spark.connector.datasource.CassandraCatalog",
            "spark.sql.catalog.comwatt.spark.cassandra.connection.compression": "LZ4",
            "spark.sql.catalog.comwatt.spark.cassandra.connection.host": "localhost:9042",
        }
    )
    .getOrCreate()
)

In [3]:
from demo.adapters.postgres_reader import PostgresClient, PostgresReader

pg_client = PostgresClient(
    spark, "jdbc:postgresql://localhost:5432/comwatt", "postgres", "demo_password"
)
PostgresReader(pg_client, "devices")().show(truncate=False)

+----------+------------------------------------+-------+---------+-------------------------+
|device_uid|connected_object_uid                |site_id|type     |label                    |
+----------+------------------------------------+-------+---------+-------------------------+
|0         |05ed9851-5343-47d9-9417-99422184eb29|0      |chauffage|Mon chauffage maison     |
|1         |0fd49130-1973-418f-b101-3682942654e0|1      |chauffage|Le chauffage c'est la vie|
|2         |14766b2f-4d6f-4849-afdf-d74639ac67a4|2      |four     |pour faire du pain       |
+----------+------------------------------------+-------+---------+-------------------------+


In [4]:
from demo.adapters.cassandra_reader import CassandraClient, CassandraReader

cass_client = CassandraClient(spark, "comwatt.comwatt")
CassandraReader(cass_client, "measures")().show(truncate=False)

+-------+-----------+------------------------------------+-------------------+----------+
|site_id|time_bucket|connected_object_uid                |timestamp          |value     |
+-------+-----------+------------------------------------+-------------------+----------+
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-12 01:00:00|106.82685 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-12 00:00:00|95.88386  |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 23:00:00|123.56678 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 22:00:00|108.77442 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 21:00:00|91.61672  |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 20:00:00|110.88063 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 19:00:00|95.758224 |
|1      |202309     |0fd49130-1973-418f-b101-3682942654e0|2023-09-11 18:00:00|95.24462  |
|1      |2